In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
#export
from exp.nb_07 import *

## ConvNet

Getting the MNIST data and a CNN

In [ ]:
x_train,y_train,x_valid,y_valid = get_data()

x_train,x_valid = normalize_to(x_train,x_valid)
train_ds,valid_ds = Dataset(x_train, y_train),Dataset(x_valid, y_valid)

nh,bs = 50,512
c = y_train.max().item()+1
loss_func = F.cross_entropy

data = DataBunch(*get_dls(train_ds, valid_ds, bs), c)

In [ ]:
mnist_view = view_tfm(1,28,28)
cbfs = [Recorder,
        partial(AvgStatsCallback,accuracy),
        CudaCallback,
        partial(BatchTransformXCallback, mnist_view)]

In [ ]:
nfs = [8,16,32,64,64]

In [ ]:
def conv_layer(ni, nf, ks=3, stride=2, **kwargs):
    return nn.Sequential(
        nn.Conv2d(ni, nf, ks, padding=ks//2, stride=stride, bias=True),
        GeneralRelu(**kwargs))

In [ ]:
learn,run = get_learn_run(nfs, data, 0.4, conv_layer, cbs=cbfs)

Here is the [original paper](https://arxiv.org/pdf/1511.06422.pdf). We initialize our neural net with the usual technique, then we pass a batch through the model and check the outputs of the linear and convolutional layers. We can then rescale the weights according to the variance we observe on the activations, and subtract the mean we observe from the initial bias. That way we will have activations that stay normalize.

We repeat this process until we are satisfied with the mean/variance we observe.

In [ ]:
def get_batch(dl, run):
    run.xb,run.yb = next(iter(dl))
    for cb in run.cbs: cb.set_runner(run)
    run('begin_batch')
    return run.xb,run.yb

In [ ]:
xb,yb = get_batch(data.train_dl, run)

We only want the outputs of convolutional or linear layers.

In [ ]:
def find_modules(m, cond):
    if cond(m): return [m]
    return sum([find_modules(o,cond) for o in m.children()], [])

In [ ]:
def is_lin_layer(l):
    lin_layers = (nn.Conv1d, nn.Conv2d, nn.Conv3d, nn.Linear)
    return isinstance(l, lin_layers)

In [ ]:
mods = find_modules(learn.model, is_lin_layer)

In [ ]:
def append_mean(hook, mod, inp, outp):
    d = outp.data
    hook.mean,hook.std = d.mean().item(),d.std().item()

In [ ]:
mdl = learn.model.cuda()

In [ ]:
with Hooks(mods,append_mean) as hooks:
    mdl(xb)
    for hook in hooks: print(hook.mean,hook.std)

0.008279498666524887 1.4534205198287964
-0.18544313311576843 1.3938395977020264
-0.03607095032930374 1.150250792503357
-0.035806965082883835 1.057843804359436
-0.04104777052998543 0.7038090229034424
-0.07098027318716049 0.26428452134132385


The idea is then to change the bias and weights accordingly to make the activations have a mean closer to 0 and a std closer to 1.

In [ ]:
def lsuv_module(m, xb):
    h = Hook(m, append_mean)

    if getattr(m, 'bias', None) is not None:
        while mdl(xb) is not None and abs(h.mean) > 1e-3:
            m.bias.data -= h.mean

    while mdl(xb) is not None and abs(h.std-1) > 1e-3:
        m.weight.data /= h.std

    h.remove()
    return h.mean,h.std

In [ ]:
for m in mods: print(lsuv_module(m, xb))

(-0.002582934917882085, 0.9999999403953552)
(-0.0059326705522835255, 1.0)
(-0.004340727347880602, 1.0000001192092896)
(-0.002693064510822296, 0.9999999403953552)
(-0.021042972803115845, 1.0)
(-0.22801573574543, 0.9995047450065613)


Then training is beginning on better grounds.

In [ ]:
%time run.fit(2, learn)

train: [1.04834015625, tensor(0.6594, device='cuda:0')]
valid: [0.1922559326171875, tensor(0.9396, device='cuda:0')]
train: [0.17182677734375, tensor(0.9472, device='cuda:0')]
valid: [0.1034960693359375, tensor(0.9676, device='cuda:0')]
CPU times: user 2.38 s, sys: 519 ms, total: 2.9 s
Wall time: 2.9 s
